In [ ]:
%matplotlib inline
import sys, os
from os.path import abspath, join, dirname
sys.path.append("../") # Adds higher directories to python modules path
import numpy as np
import evaluator, agent, env
import time
import progressbar

## Overview

In this tutorial, we will define a 2D reaching task and evaluate two model based agents. The two agents represent the robot and the human separately. 

We first define the testing environment with `env_spec`, and then define the computational models (model based agents) that give the control signals with `agents`.

In the end, we instantiate this environment with the `env_spec` and `agents`.


## Environment

The environment is fully characterized by `env_spec`. `env_spec` contains `world`, `dt` and `agent_env_spec`.

`world` specifies which physics engine we use, here we use a 2D physics engine.

`dt` is the time sampling separation.

`agent_env_spec` specifies the physical agents (car, robot arm, ball, etc) that will be controlled by the computational model.

### Physical Agent

We will use two BB8 agents, which are essentially balls. 

`agent_env_spec`: A dictionary, the key is the name of the physical agent, and the value is the specification. 

`type`: The type of the physical agent, see "env/flat_world/agent.py" for all available agents. 

`spec` The parameters required by the physical agent for initialization. For the `BB8Agent`, we only need to provide the init state, which is `[x, y, vx, vy]`.


In [ ]:
agent_env_spec = {"robot":{"type":"BB8Agent", "spec":{"init_x":np.vstack([ 30.,20.0, 0., 0.])}},
                  "human":{"type":"BB8Agent", "spec":{"init_x":np.vstack([ 50.,20.0, 0., 0.])}}
                }


### World

We use a 2D physics engine, which we call `FlatReachingWorld`. This engine requires some parameters which we specified in `reaching_world_spec`.

`friction`: Friction force.

`reaching_eps`: $L_\infty$ tolerance to decide whether the agent reaches the goal.

`agent_goal_lists`: Goals for each agent, once the agent reaches a goal, this goal will disapper and the next goal will show up.

In [ ]:
reaching_world_spec = {
    "friction": 0,
    "reaching_eps": 0.1,
    "agent_goal_lists":{
        "robot": [[70.0,20.0], [10, 40]],
        "human": [[10.0,20.0], [40, 70]],
    }
}

Then we define the `env_spec`.

In [ ]:
dt = 0.02
env_spec = {
    "world": {"type":"FlatReachingWorld", "spec":reaching_world_spec},
    "dt": dt,
    "agent_env_spec": agent_env_spec
}

In the next, we define the computational model for each physical agent. We use model-based agents as computational models.

## Model-based Agent

We define a model based agent by specifying the `task`, `estimator`, `planner`, `controller` and `sensors`. The specification of each module is composed by two parts, `type` and `spec`.

### Task

The task module tells the agent what to do by giving the agent a reference goal computed from the observations. Here we define a FlatReachingTask. This task asks the agent to reach a point in the environment.

In [ ]:
task_spec = {"type":"FlatReachingTask",    "spec":{}}

### Model
The model module contains structured differentiable functions that either represent system dynamics or control policies. This module will be used by estimator, planner and controller.

In [ ]:
model_spec = {"type":"LinearModel",     "spec":{"use_spec":0, 
                                                "use_library":0, 
                                                "model_name":'Ballbot', 
                                                "time_sample":dt, 
                                                "disc_flag":1, 
                                                "model_spec":None,
                                                "control_input_dim":2}}

### Estimator

The estimator module contains a class of functions that filt raw observations and estimate agent state. Here we define a UKF estimator. 

In [ ]:
estimator_spec = {"type":"EKFEstimator",   "spec":{"init_x":np.array([ 30.,20.0, 0., 0.]),
                                                   "init_variance":.01*np.eye(4),
                                                   "Rww":.001*np.eye(4),
                                                   "Rvv":.001*np.eye(4),
                                                   "alpha_ukf":1,
                                                   "kappa_ukf":0.1,
                                                   "beta_ukf":2,
                                                   "time_sample":dt,
                                                   "kp":6,
                                                   "kv":8}}

### Planner
The  planner  module  contains  a  class  of  functions  that  map  observation  to  asequence  of  future  states  of  future  inputs. Here we define a optimization based planner. This planner will plan a trajectory based on current location and the goal provided by the task module.

In [ ]:
planner_spec = {"type":"OptimizationBasedPlanner","spec":{"horizon":10, 
                                                          "replanning_cycle":10, 
                                                          "dim":2, 
                                                          "n_ob":1, 
                                                          "obs_r":5}}

### Controller

The controller module contains a class of functions that map observation to thenext control input. Here we define a PID controller.

In [ ]:
controller_spec = {"type":"NaiveController", "spec":{"kp":6,
                                                     "kv":8}}

### Sensor

The sensor module defines the sensors the agent equiped. An agent can have multiple sensors. Each sensor has an alias. The alias tells us what this sensor is used for. Because there are multiple sensors can do the same work. For short, `alias` describe the purpose of the sensor, and `type` describe the function.

Check the "env/XXX_worlds/sensors" to see avaiable sensors.

In [ ]:
sensors_spec = [{"type":"PVSensor",                "spec":{"alias":"cartesian_sensor","noise_var":0.1}},
                {"type":"StateSensor",             "spec":{"alias":"state_sensor",    "noise_var":0.1}},
                {"type":"RadarSensor",             "spec":{"alias":"obstacle_sensor", "noise_var":0.1}},
                {"type":"GoalSensor",              "spec":{"alias":"goal_sensor",     "noise_var":0.0}},
                {"type":"RadioSensor",             "spec":{"alias":"communication_sensor"}},
               ]

### Assemble Agent

Now we assemble the agent by combining all these modules. And give the agent a name as "robot".

In [ ]:
agent1_module_spec = {
        "name":      "robot",
        "task":      task_spec,
        "model":     model_spec,
        "estimator": estimator_spec,
        "planner":   planner_spec,
        "controller":controller_spec,
        "sensors":   sensors_spec,
}


### Agents

We define a similar model-based agent and name it as "human". Then we instantiate these two agents.

In [ ]:
agent2_module_spec = {
        "name":      "human",
        "task":      task_spec,
        "model":     model_spec,
        "estimator": estimator_spec,
        "planner":   planner_spec,
        "controller":controller_spec,
        "sensors":   sensors_spec,
}

agent_specs = [agent1_module_spec, agent2_module_spec] # specs for two agents

agents = []
for i in range(len(agent_specs)):
    agents.append(agent.ModelBasedAgent(agent_specs[i]))

Instantiate the environment.

In [ ]:
e = env.FlatEnv(env_spec, agents)

Evaluate the agents step by step. In each step, the agent receives observation from the environment and gives action.

In [ ]:
dt, env_info, measurement_groups = e.reset()
record = []
print("Simulation progress:")
for it in progressbar.progressbar(range(1000)):
    actions = {}
    for agent in agents:
        # an action is dictionary which must contain a key "control"
        actions[agent.name] = agent.action(dt, measurement_groups[agent.name])
        #sensor data is grouped by agent
    dt, env_info, measurement_groups = e.step(actions, render = (it%50 == 0))
    record.append(env_info)

ev = evaluator.Evaluator(agent_specs, env_spec)
ev.evaluate(record)